In [1]:
import json
import random
from serpapi import GoogleSearch
import discord
from discord.ext.commands import Bot

import speech_recognition as sr

In [2]:
bot = Bot(command_prefix='$')

# Global Variables
bot.show = False

@bot.event
async def on_ready():
    print(f'Bot connected as {bot.user}')

@bot.event
async def on_message(message):
    if not message.author == bot.user:
        msg = message.content.lower()
        if msg.startswith("test"):
            await message.channel.send('Hey there!')

        if msg.startswith('show') and not bot.show:
            await message.channel.send('Let\'s see what we have here')
            await bot.change_presence(activity=discord.Game(name="the listen to Larry game"))
            bot.show = True
        elif msg.startswith('quit') and bot.show:
            await message.channel.send('Time to stop playing around.')
            await bot.change_presence()
            bot.show = False
        elif bot.show:
            print(f"Searching for: {msg}")
            url = processQuery(msg)
            await message.channel.send(url)

def getThumbnails(query):
    params = {
        "q": query,
        "tbm": "isch",
        "ijn": "0",
        "api_key": "3c77c2fec20389dc34133f3604683c81573fcaa85b04cbb67f2ac73f29a8089d"
    }
    #return "key", ["val1","val2"]

    search = GoogleSearch(params)
    results = search.get_dict()
    images_results = results['images_results']
    print(f"\t{len(images_results)} images sourced and stored")
    return query, [r['thumbnail'] for r in images_results]


def writeJson(res):
    global filename, data
    with open(filename, "r+") as file:
        temp = json.load(file)
        temp[res[0]] = res[1]
        file.seek(0)
        json.dump(temp, file, indent = 4)
        data = temp

def processQuery(query):
    query = query.strip().lower()

    global data
    if not data:
        with open(filename, "r") as file:
            data = json.load(file)
    if query not in data:

        # Remove (backup safety key)
        print("Cancelling")
        return "I don't know what that looks like"

        print("\tUnrecognized Query: calling API")
        writeJson(getThumbnails(query))
    return random.choice(data[query])

# Global variables - May need refactoring
filename = "data.json"
with open(filename, "r") as file:
    data = json.load(file)

# Token for PythonTestBot#9025
bot.run("ODQxNzY5NTYxMTUyOTQ2MTc3.YJrk_g.7eZajyAG7hkeUdyhGj03H2usBZo")

RuntimeError: Cannot close a running event loop